# Scope: Trecerea de la problema la solutie

In [52]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
import openai
import json
# Helps asyncio run within Jupyter
import nest_asyncio

import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored


load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

if os.getenv("OPENAI_API_KEY") is not None:
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")

GPT_MODEL = "gpt-3.5-turbo-16k"

OPENAI_API_KEY is ready


In [53]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, temperature=0, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages, "temperature": temperature}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [54]:
# Read data about the product

with open('/Users/vladbordei/Documents/Development/ProductExplorer/data/interim/short_product_information.json') as file:
    json_string = file.read()
    general_product_data = json.loads(json_string)

In [55]:
#asin_list_path = './data/external/asin_list.csv'
asin_list_path = '/Users/vladbordei/Documents/Development/ProductExplorer/data/external/asin_list.csv'
asin_list = pd.read_csv(asin_list_path)['asin'].tolist()

In [56]:
attribute_clusters_with_percentage_path = '/Users/vladbordei/Documents/Development/ProductExplorer/data/interim/attribute_clusters_with_percentage.csv'
attributes = pd.read_csv(attribute_clusters_with_percentage_path)

In [57]:
reviews_with_clusters_path = '/Users/vladbordei/Documents/Development/ProductExplorer/data/interim/reviews_with_clusters.csv'
reviews_with_clusters = pd.read_csv(reviews_with_clusters_path)

In [58]:
df_problem_statements_path = '/Users/vladbordei/Documents/Development/ProductExplorer/data/interim/problem_statements.csv'
df_problem_statements = pd.read_csv(df_problem_statements_path)

In [59]:
attributes_data = attributes[['Attribute', 'cluster_label', 'positive_sentiment',
       'negative_sentiment','observation_count', 'rating_avg', 'percentage_of_observations_vs_total_number_per_attribute','percentage_of_observations_vs_total_number_of_reviews' ]]

In [60]:
general_product_data

{'Product Summary': 'The Raised Garden Bed is a 6x3x2ft galvanized planter box designed for growing vegetables, flowers, and herbs. It is made of stable thickened steel with an evaluated coating paint, ensuring long-term use without rusting. The open bottom design allows for good drainage, promoting healthy plant growth. Setting up the garden bed is quick and easy, as it does not require any tools. The butterfly screws provided can be fastened by hand in just 5-10 minutes. The garden bed also features a cured edge for safety, preventing any injuries to your hands.',
 'In_the_Box': 'The package includes the FRIZIONE Galvanized Metal Raised Garden Bed and all the necessary hardware for assembly.',
 'technical_facts': 'The garden bed is made of galvanized steel with an anti-rust coating. It has a 2ft depth and measures 6x3ft in size.',
 'features': '1. Size: 6x3x2ft, providing ample growing space for vegetables, herbs, and flowers.\n2. Made of stable thickened steel with evaluated coating

In [68]:
# De redus in dimensionalitate pentru a putea fi folosit

attribute_summary = attributes.groupby('Attribute').agg(
    Total_Observations=('observation_count', 'sum'),
    Average_Positive_Sentiment=('positive_sentiment', 'mean'),
    Average_Negative_Sentiment=('negative_sentiment', 'mean'),
    Average_Rating=('rating_avg', 'mean')
).reset_index()

# Clusters with the highest positive sentiment
top_pos_sentiment = attributes.nlargest(3, 'positive_sentiment')

# Clusters with the highest negative sentiment
top_neg_sentiment = attributes.nlargest(3, 'negative_sentiment')

# Clusters with the most observations
most_observations = attributes.nlargest(3, 'observation_count')

# Clusters with the least observations
least_observations = attributes.nsmallest(3, 'observation_count')

highest_avg_pos_sentiment_attribute = attribute_summary.loc[attribute_summary['Average_Positive_Sentiment'].idxmax()]['Attribute']
highest_avg_neg_sentiment_attribute = attribute_summary.loc[attribute_summary['Average_Negative_Sentiment'].idxmax()]['Attribute']

observations_dict = {}
# observations_dict['attribute_summary'] = attribute_summary.to_dict('records')
observations_dict['top_pos_sentiment'] = top_pos_sentiment.to_dict('records')
observations_dict['top_neg_sentiment'] = top_neg_sentiment.to_dict('records')
observations_dict['most_observations'] = most_observations.to_dict('records')
observations_dict['least_observations'] = least_observations.to_dict('records')
observations_dict['highest_avg_pos_sentiment_attribute'] = highest_avg_pos_sentiment_attribute
observations_dict['highest_avg_neg_sentiment_attribute'] = highest_avg_neg_sentiment_attribute

{'top_pos_sentiment': [{'Unnamed: 0': 23,
   'Attribute': 'Danger Appraisal',
   'cluster_label': 'safe for children',
   'positive_sentiment': 0.9901695847511292,
   'negative_sentiment': 0.0026110601611435,
   'Rating': '[5]',
   'id': '[16]',
   'asin': "['B09SQZJV67']",
   'URL': "['https://www.amazon.com/gp/customer-reviews/RBOQPOP0PXSCO/ref=cm_cr_getr_d_rvw_ttl?ie=UTF8&ASIN=B09SQZJV67']",
   'observation_count': 1,
   'rating_avg': 5,
   'percentage_of_observations_vs_total_number_per_attribute': 3.7037037037037033,
   'percentage_of_observations_vs_total_number_of_reviews': 0.4545454545454545},
  {'Unnamed: 0': 51,
   'Attribute': 'Noise and Smell',
   'cluster_label': 'Creates a soothing sound',
   'positive_sentiment': 0.990017294883728,
   'negative_sentiment': 0.0051168049685657,
   'Rating': '[5]',
   'id': '[190]',
   'asin': "['B0BJZ9J4HD']",
   'URL': "['https://www.amazon.com/gp/customer-reviews/R3MULGZJ4C473G/ref=cm_cr_getr_d_rvw_ttl?ie=UTF8&ASIN=B0BJZ9J4HD']",
   'obs

In [62]:
product_improvement_function = [
    {
        "name": "product_improvement_function",
        "description": "This function is designed to provide engineering solutions to address the primary issues with a product. The function uses the data from customer complaints and the product's description to propose technical product improvements. The Implementation Details  should be concise, yet comprehensive, explaining the rationale behind the solution and step-by-step instructions for carrying out the implementation. It should not contain jargon or technical terms that are not commonly understood by engineers in the relevant field.",
        "parameters": {
            "type": "object",
            "properties": {
                "Product Improvement 1": {
                    "type": "object",
                    "properties": {
                        "Title": {
                            "type": "string",
                            "description": "The title or short description of the first proposed product improvement."
                        },
                        "Implementation Details for the engineer": {
                            "type": "string",
                            "description": "A detailed, 200-word description of the first improvement solution, including specific instructions for implementation by an engineer."
                        },
                        "Considerations": {
                            "type": "string",
                            "description": "Considerations and potential challenges in the implementation of the proposed solution."
                        }
                    },
                    "required": ["Title", "Implementation Details for the engineer", "Considerations"]
                },
                "Product Improvement 2": {
                    "type": "object",
                    "properties": {
                        "Title": {
                            "type": "string",
                            "description": "The title or short description of the second proposed product improvement."
                        },
                        "Implementation Details for the engineer": {
                            "type": "string",
                            "description": "A detailed, 200-word description of the second improvement solution, including specific instructions for implementation by an engineer."
                        },
                        "Considerations": {
                            "type": "string",
                            "description": "Considerations and potential challenges in the implementation of the proposed solution."
                        }
                    },
                    "required": ["Title", "Implementation Details for the engineer", "Considerations"]
                },
                "Product Improvement 3": {
                    "type": "object",
                    "properties": {
                        "Title": {
                            "type": "string",
                            "description": "The title or short description of the third proposed product improvement."
                        },
                        "Implementation Details for the engineer": {
                            "type": "string",
                            "description": "AA detailed, 200-word description of the third improvement solution, including specific instructions for implementation by an engineer."
                        },
                        "Considerations": {
                            "type": "string",
                            "description": "Considerations and potential challenges in the implementation of the proposed solution."
                        }
                    },
                    "required": ["Title", "Implementation Details for the engineer", "Considerations"]
                },
                "Product Improvement 4": {
                    "type": "object",
                    "properties": {
                        "Title": {
                            "type": "string",
                            "description": "The title or short description of the fourth proposed product improvement."
                        },
                        "Implementation Details for the engineer": {
                            "type": "string",
                            "description": "A detailed, 200-word description of the fourth improvement solution, including specific instructions for implementation by an engineer."
                        },
                        "Considerations": {
                            "type": "string",
                            "description": "Considerations and potential challenges in the implementation of the proposed solution."
                        }
                    },
                    "required": ["Title", "Implementation Details for the engineer", "Considerations"]
                },
                "Product Improvement 5": {
                    "type": "object",
                    "properties": {
                        "Title": {
                            "type": "string",
                            "description": "The title or short description of the fifth proposed product improvement."
                        },
                        "Implementation Details for the engineer": {
                            "type": "string",
                            "description": "A detailed, 200-word description of the fifth improvement solution, including specific instructions for implementation by an engineer."
                        },
                        "Considerations": {
                            "type": "string",
                            "description": "Considerations and potential challenges in the implementation of the proposed solution."
                        }
                    },
                    "required": ["Title", "Implementation Details for the engineer", "Considerations"]
                },
                "Product Improvement 6": {
                    "type": "object",
                    "properties": {
                        "Title": {
                            "type": "string",
                            "description": "The title or short description of the sixth proposed product improvement."
                        },
                        "Implementation Details for the engineer": {
                            "type": "string",
                            "description": "A detailed, 200-word description of the sixth improvement solution, including specific instructions for implementation by an engineer."
                        },
                        "Considerations": {
                            "type": "string",
                            "description": "Considerations and potential challenges in the implementation of the proposed solution."
                        }
                    },
                    "required": ["Title", "Implementation Details for the engineer", "Considerations"]
                },
                "Product Improvement 7": {
                    "type": "object",
                    "properties": {
                        "Title": {
                            "type": "string",
                            "description": "The title or short description of the seventh proposed product improvement."
                        },
                        "Implementation Details for the engineer": {
                            "type": "string",
                            "description": "A detailed, 200-word description of the seventh improvement solution, including specific instructions for implementation by an engineer."
                        },
                        "Considerations": {
                            "type": "string",
                            "description": "Considerations and potential challenges in the implementation of the proposed solution."
                        }
                    },
                    "required": ["Title", "Implementation Details for the engineer", "Considerations"]
                },
            },
            "required": ["Product Improvement 1", "Product Improvement 2", "Product Improvement 3", "Product Improvement 4", "Product Improvement 5", "Product Improvement 6", "Product Improvement 7"]
        }
    }
]



In [63]:
product_sustainability_function = [
    {
        "name": "product_sustainability_function",
        "description": "This function is designed to provide economically feasible and sustainability enhancing solutions for a product. The function uses data from customer feedback, product description, and environmental impact analysis to propose product improvements. The Implementation Details should be concise and comprehensive, explaining the rationale behind the solution and step-by-step instructions for carrying out the implementation. It should avoid jargon or technical terms not commonly understood by engineers in the relevant field.",
        "parameters": {
            "type": "object",
            "properties": {
                "Sustainable Improvement 1": {
                    "type": "object",
                    "properties": {
                        "Title": {
                            "type": "string",
                            "description": "The title or short description of the first proposed sustainable improvement."
                        },
                        "Implementation Details for the engineer": {
                            "type": "string",
                            "description": "A detailed, 200-word description of the first sustainability solution, including specific instructions for implementation by an engineer."
                        },
                        "Economic Feasibility": {
                            "type": "string",
                            "description": "A brief analysis of the economic feasibility of the proposed solution."
                        },
                        "Considerations": {
                            "type": "string",
                            "description": "Considerations and potential challenges in the implementation of the proposed solution."
                        }
                    },
                    "required": ["Title", "Implementation Details for the engineer", "Economic Feasibility", "Considerations"]
                },
                "Sustainable Improvement 2": {
                    "type": "object",
                    "properties": {
                        "Title": {
                            "type": "string",
                            "description": "The title or short description of the second proposed sustainable improvement."
                        },
                        "Implementation Details for the engineer": {
                            "type": "string",
                            "description": "A detailed, 200-word description of the second sustainability solution, including specific instructions for implementation by an engineer."
                        },
                        "Economic Feasibility": {
                            "type": "string",
                            "description": "A brief analysis of the economic feasibility of the proposed solution."
                        },
                        "Considerations": {
                            "type": "string",
                            "description": "Considerations and potential challenges in the implementation of the proposed solution."
                        }
                    },
                    "required": ["Title", "Implementation Details for the engineer", "Economic Feasibility", "Considerations"]
                },
                "Sustainable Improvement 3": {
                    "type": "object",
                    "properties": {
                        "Title": {
                            "type": "string",
                            "description": "The title or short description of the third proposed sustainable improvement."
                        },
                        "Implementation Details for the engineer": {
                            "type": "string",
                            "description": "A detailed, 200-word description of the third sustainability solution, including specific instructions for implementation by an engineer."
                        },
                        "Economic Feasibility": {
                            "type": "string",
                            "description": "A brief analysis of the economic feasibility of the proposed solution."
                        },
                        "Considerations": {
                            "type": "string",
                            "description": "Considerations and potential challenges in the implementation of the proposed solution."
                        }
                    },
                    "required": ["Title", "Implementation Details for the engineer", "Economic Feasibility", "Considerations"]
                },
                "Sustainable Improvement 4": {
                    "type": "object",
                    "properties": {
                        "Title": {
                            "type": "string",
                            "description": "The title or short description of the fourth proposed sustainable improvement."
                        },
                        "Implementation Details for the engineer": {
                            "type": "string",
                            "description": "A detailed, 200-word description of the fourth sustainability solution, including specific instructions for implementation by an engineer."
                        },
                        "Economic Feasibility": {
                            "type": "string",
                            "description": "A brief analysis of the economic feasibility of the proposed solution."
                        },
                        "Considerations": {
                            "type": "string",
                            "description": "Considerations and potential challenges in the implementation of the proposed solution."
                        }
                    },
                    "required": ["Title", "Implementation Details for the engineer", "Economic Feasibility", "Considerations"]
                },
                "Sustainable Improvement 5": {
                    "type": "object",
                    "properties": {
                        "Title": {
                            "type": "string",
                            "description": "The title or short description of the fifth proposed sustainable improvement."
                        },
                        "Implementation Details for the engineer": {
                            "type": "string",
                            "description": "A detailed, 200-word description of the fifth sustainability solution, including specific instructions for implementation by an engineer."
                        },
                        "Economic Feasibility": {
                            "type": "string",
                            "description": "A brief analysis of the economic feasibility of the proposed solution."
                        },
                        "Considerations": {
                            "type": "string",
                            "description": "Considerations and potential challenges in the implementation of the proposed solution."
                        }
                    },
                    "required": ["Title", "Implementation Details for the engineer", "Economic Feasibility", "Considerations"]
                }
            },
            "required": ["Sustainable Improvement 1", "Sustainable Improvement 2", "Sustainable Improvement 3", "Sustainable Improvement 4", "Sustainable Improvement 5"]
        }
    }
]


In [75]:
problem = df_problem_statements['problem_statement']
messages = [
    {"role": "user", "content": f"PROBLEM STATEMENT: {problem} ,PRODUCT DESCRIPTION: {general_product_data} , ANALYSIS: {attributes_data}"}
]

# Send the request to the LLM and get the response
response =  chat_completion_request(
    messages=messages,
    functions= product_improvement_function,
    function_call={"name": "product_improvement_function"},
    temperature=0.5,
    model=GPT_MODEL
)

# Process the response and store in the dictionary
chat_response = response.json()["choices"][0]
print(chat_response)

{'index': 0, 'message': {'role': 'assistant', 'content': None, 'function_call': {'name': 'product_improvement_function', 'arguments': '{\n  "Product Improvement 1": {\n    "Title": "Enhance Durability and Sturdiness",\n    "Implementation Details for the engineer": "1. Increase the thickness of the galvanized steel used in the construction of the garden bed to improve its durability.\\n2. Reinforce the corners and joints of the garden bed with additional welding or fasteners to enhance its sturdiness.\\n3. Use high-quality materials for the hardware components to ensure long-lasting performance.\\n4. Conduct thorough testing and quality control measures during the manufacturing process to identify and address any potential weaknesses or defects.\\n5. Consider incorporating additional support structures, such as crossbars or braces, to further strengthen the garden bed.\\n6. Apply a protective coating or finish to the steel surface to prevent rusting and corrosion.\\n7. Provide clear in

In [76]:
data = eval(chat_response['message']['function_call']['arguments'])

SyntaxError: unterminated string literal (detected at line 9) (<string>, line 9)

In [74]:
data

{'Product Improvement 1': {'Title': 'Increase Durability',
  'Implementation Details for the engineer': '1. Use a thicker gauge of galvanized steel for the construction of the garden bed to increase its strength and durability.\n2. Apply multiple layers of anti-rust coating to enhance the longevity of the product.\n3. Reinforce the corners and edges of the garden bed with additional welding or bracing to prevent bending or warping.\n4. Conduct thorough quality control checks during the manufacturing process to ensure that the materials and construction meet high standards.\n5. Provide clear instructions for proper assembly and maintenance to ensure that the garden bed is used correctly.',
  'Considerations': '1. The increased thickness of the steel may slightly increase the weight of the garden bed.\n2. The multiple layers of coating may increase the cost of production.\n3. Additional welding or bracing may require additional manufacturing steps and costs.\n4. Clear instructions may ne

In [77]:
problem = df_problem_statements['problem_statement']
messages = [
    {"role": "user", "content": f"PROBLEM STATEMENT: {problem} ,PRODUCT DESCRIPTION: {general_product_data} , ANALYSIS: {attributes_data}"}
]

# Send the request to the LLM and get the response
sustainability_response =  chat_completion_request(
    messages=messages,
    functions= product_sustainability_function,
    function_call={"name": "product_sustainability_function"},
    temperature=0.5,
    model=GPT_MODEL
)

# Process the response and store in the dictionary
sustainability_chat_response = sustainability_response.json()["choices"][0]
print(sustainability_chat_response)

{'index': 0, 'message': {'role': 'assistant', 'content': None, 'function_call': {'name': 'product_sustainability_function', 'arguments': '{\n  "Sustainable Improvement 1": {\n    "Title": "Increase Durability",\n    "Implementation Details for the engineer": "1. Use higher quality materials for the construction of the garden bed, such as stainless steel or treated wood, to improve durability.\\n2. Apply a stronger coating or paint to protect against rust and corrosion.\\n3. Reinforce the edges and corners of the garden bed to prevent bending or warping.\\n4. Conduct rigorous testing to ensure the garden bed can withstand various weather conditions and heavy use.\\n5. Provide clear instructions for proper assembly and maintenance to prolong the lifespan of the product.",\n    "Economic Feasibility": "Investing in higher quality materials and coatings may increase the production cost of the garden bed. However, it can lead to higher customer satisfaction and repeat purchases, ultimately 

In [78]:
sustainable_data = eval(sustainability_chat_response['message']['function_call']['arguments'])

In [79]:
sustainable_data

{'Sustainable Improvement 1': {'Title': 'Increase Durability',
  'Implementation Details for the engineer': '1. Use higher quality materials for the construction of the garden bed, such as stainless steel or treated wood, to improve durability.\n2. Apply a stronger coating or paint to protect against rust and corrosion.\n3. Reinforce the edges and corners of the garden bed to prevent bending or warping.\n4. Conduct rigorous testing to ensure the garden bed can withstand various weather conditions and heavy use.\n5. Provide clear instructions for proper assembly and maintenance to prolong the lifespan of the product.',
  'Economic Feasibility': "Investing in higher quality materials and coatings may increase the production cost of the garden bed. However, it can lead to higher customer satisfaction and repeat purchases, ultimately improving the product's economic viability.",
  'Considerations': 'Consider the additional cost of using higher quality materials and coatings. Conduct market

{'index': 0, 'message': {'role': 'assistant', 'content': None, 'function_call': {'name': 'product_improvement_function', 'arguments': '{\n  "Product Improvement 1": {\n    "Title": "Increase Durability",\n    "Implementation Details for the engineer": "To increase the durability of the Raised Garden Bed, the following steps can be taken:\\n1. Use a thicker gauge galvanized steel for the construction of the planter box. This will provide added strength and resistance to bending or warping.\\n2. Apply a double layer of anti-rust coating to the steel. This will enhance the protective barrier against corrosion and extend the lifespan of the garden bed.\\n3. Reinforce the corners and edges of the planter box with additional welding or bracing. This will prevent any weak points and improve overall structural integrity.\\n4. Use high-quality butterfly screws for assembly. These screws should be made of durable materials, such as stainless steel, to ensure they do not rust or degrade over time

In [124]:
data = eval(chat_response['message']['function_call']['arguments'])

In [125]:
data

{'Product Improvement 1': {'Title': 'Increase Durability',
  'Implementation Details for the engineer': 'To increase the durability of the Raised Garden Bed, the following steps can be taken:\n1. Use a thicker gauge galvanized steel for the construction of the planter box. This will provide added strength and resistance to bending or warping.\n2. Apply a double layer of anti-rust coating to the steel. This will enhance the protective barrier against corrosion and extend the lifespan of the garden bed.\n3. Reinforce the corners and edges of the planter box with additional welding or bracing. This will prevent any weak points and improve overall structural integrity.\n4. Use high-quality butterfly screws for assembly. These screws should be made of durable materials, such as stainless steel, to ensure they do not rust or degrade over time.\n5. Conduct thorough quality control checks during the manufacturing process to identify and address any potential weaknesses or defects.\n6. Provide 

In [37]:
set(attributes_data['Attribute'])


{'Buyer Motivation',
 'Colors',
 'Customer Expectations',
 'Danger Appraisal',
 'Design and Appearance',
 'Durability',
 'Ease of Use',
 'How the product is used',
 'Issues',
 'Noise and Smell',
 'Packaging',
 'Parts and Components',
 'Price',
 'Quality',
 'Season',
 'Setup and Instructions',
 'Size and Fit',
 'User Description',
 'When the product is used',
 'Where the product is used'}

In [38]:
attributes_data[attributes_data['Attribute'] == 'Buyer Motivation']

,Attribute,cluster_label,positive_sentiment,negative_sentiment,observation_count,rating_avg,percentage_of_observations_vs_total_number_per_attribute,percentage_of_observations_vs_total_number_of_reviews
0,Buyer Motivation,Buyer Motivation,0.472605,0.272846,16,4,12.307692,7.272727
1,Buyer Motivation,Motivation for gardening and garden decoration,0.737425,0.123885,23,4,17.692308,10.454545
2,Buyer Motivation,Motivation to have raised garden beds,0.553676,0.240755,25,4,19.230769,11.363636
3,Buyer Motivation,Motivation to meet specific needs and preferences,0.522197,0.315513,30,4,23.076923,13.636364
4,Buyer Motivation,Motivation to start a garden,0.853987,0.072101,30,5,23.076923,13.636364
5,Buyer Motivation,To try out a new product,0.435807,0.343664,6,4,4.615385,2.727273


In [39]:
attributes_data[attributes_data['Attribute'] == 'Colors']

,Attribute,cluster_label,positive_sentiment,negative_sentiment,observation_count,rating_avg,percentage_of_observations_vs_total_number_per_attribute,percentage_of_observations_vs_total_number_of_reviews
6,Colors,Color options and preferences,0.908285,0.007926,3,5,13.636364,1.363636
7,Colors,Limited color options,0.638209,0.221642,6,4,27.272727,2.727273
8,Colors,Neutral color options,0.850350,0.028127,3,5,13.636364,1.363636
9,Colors,Painted and Galvanized with a brown color,0.588862,0.142078,2,4,9.090909,0.909091
10,Colors,Unknown,0.320234,0.641082,3,3,13.636364,1.363636
11,Colors,Variety of finishes and colors available,0.688907,0.056164,3,4,13.636364,1.363636
12,Colors,White and Blue Colors,0.939245,0.006660,2,5,9.090909,0.909091


In [11]:
attributes_data['observation_count'].sum()

1541

In [40]:
attributes_data.sort_values(by='percentage_of_observations_vs_total_number_of_reviews', ascending=False)

,Attribute,cluster_label,positive_sentiment,negative_sentiment,observation_count,rating_avg,percentage_of_observations_vs_total_number_per_attribute,percentage_of_observations_vs_total_number_of_reviews
13,Customer Expectations,Customer Expectations,0.579193,0.256274,73,4,56.589147,33.181818
34,Ease of Use,Ease of Use,0.771226,0.096421,59,4,64.130435,26.818182
24,Design and Appearance,Design and Appearance,0.753909,0.113694,50,5,86.206897,22.727273
80,Quality,High quality products with good and great quality,0.831325,0.075905,41,5,48.809524,18.636364
45,Issues,Flimsy and unstable product with various issues,0.201460,0.556913,36,3,26.666667,16.363636
...,...,...,...,...,...,...,...,...
52,Noise and Smell,No issues,0.006217,0.851541,1,1,3.846154,0.454545
51,Noise and Smell,Creates a soothing sound,0.990017,0.005117,1,5,3.846154,0.454545
54,Noise and Smell,No specific observations on the noise level or...,0.671206,0.074277,1,4,3.846154,0.454545
18,Danger Appraisal,Not applicable,0.893003,0.004798,1,5,3.703704,0.454545
